# Cold Email Generator 

This notebook contains a cold email generator for the [PerlonAI](https://www.perlonai.com/) take home project. 

[Instructions](https://docs.google.com/document/d/1fh5xpp0yrpuyohcQRg6OqPfI7H4XDXIaedpz--QH7rs/edit)

[Example Data](https://docs.google.com/spreadsheets/d/1FoLsD7hE-ciClZf-nTDsxCDMqmaXg5gOhZ5Z8_ozaEo/edit#gid=0)

The notebook is split into the following sections:
- Setup
- Experimentation
- Further Improvements

## Setup

In [3]:
! pip freeze > requirements.txt

In [14]:
from openai import OpenAI
from dotenv import load_dotenv
import os
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import anthropic
from tqdm import tqdm_notebook as tqdm
import tiktoken
import statistics
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_community.chat_models import ChatOpenAI
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain import LLMChain

In [12]:
load_dotenv()
client = OpenAI()
anthropic_client = anthropic.Anthropic()

## Experimentation

In [15]:
#Testing OpenAI calls
chat_completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Hello world"}]
)
res = chat_completion.choices[0].message.content
print(res)

Hello! How can I assist you today?


### Writing PerlonAI Company Description
- self-generated
- llm-assisted from website

In [4]:
#Wrote short company description myself
self_description = """
PerlonAI is a company that specialises in the automated generation of cold outbound emails. 
PerlonAI obtains relevant contextual information from the internet and personalises emails through generative AI.
"""

In [16]:
#Making request from website
fp = urllib.request.urlopen("https://www.perlonai.com/")
mybytes = fp.read()

html_doc = mybytes.decode("utf8")
fp.close()

# print(html_doc)


In [17]:
#Getting only text from website
soup = BeautifulSoup(html_doc, 'html.parser')
website_content = soup.get_text()
# print(website_content)

In [18]:
prompt = """
You are an expert at parsing and extracting information from websites. 
You will recieve the extracted content from a website.
Summarise this content, extracting key information about the company, statistics and business offerings.
"""
chat_completion = client.chat.completions.create(
    model="gpt-4-turbo-2024-04-09",
    messages=[{"role":"system", "content": prompt, "role": "user", "content": f"Can you please summarise: {website_content}"}]
)
perlon_description = chat_completion.choices[0].message.content
print(perlon_description)

PerlonAI specializes in sending hyper-personalized cold emails at scale using customized AI models. These AI models conduct extensive research, tailor emails to individual prospects, and manage the sending process, promising 5X more replies compared to standard templates.

Key features include:
1. Automation of the entire email campaign process—from research and writing to sending—utilizing a custom AI language model that adapts to a company’s unique brand voice.
2. Significant time savings, allowing users to bypass manual crafting of personalized emails, which are both time-consuming and less scalable.
3. Avoidance of spam filters due to the unique nature of each email, leading to higher open and engagement rates.

The system integrates directly with user mailboxes for seamless operation and can be live within three days of setup. Users can expect a dramatic increase in email campaign volume and effectiveness without sacrificing the personal touch critical in reaching prospects.

Perl

In [8]:
# Saved generated description from above
perlon_description = """PerlonAI specializes in the creation and distribution of automated, hyper-personalized cold emails designed to significantly improve engagement and response rates over standard email templates. The company employs sophisticated AI models tailored to each customer's unique brand voice and requirements. These AI models automate the entire process, including researching, crafting, and sending emails directly from the user's mailbox.

Key benefits reported by PerlonAI include:
1. **Increased Efficiency**: Template-based campaigns are statistically ineffective; PerlonAI enables personalization at scale without the extensive time investment traditionally required to craft personalized emails, claiming to provide 500% more replies and an 86% increase in email opens.
2. **Time Savings**: It reduces the time spent on drafting emails from 15-30 minutes each to essentially zero, giving sales teams more time to focus on closing deals.
3. **Spam Filter Navigation**: Because each email is unique, they are more likely to bypass spam filters effectively.
4. **Enhanced Email Volume**: The system supports sending a significantly higher volume of emails compared to manual drafting.

The process includes a consultation to understand client objectives, model building which leverages real-time data, and integration directly with the client's existing mailboxes to start campaigns effectively within three days.

Prospective clients are encouraged to book a demo to see how PerlonAI can help automate their email prospecting efforts, aiming to transform how companies conduct their outreach by merging automation with personalization for superior results."""

### Few-shot prompting
- example cold emails from PerlonAI website
- few-shot prompting 
- OpenAI vs Anthropic

In [6]:
# Pulled example emails from the website 
example_email_1 = """Hi Addie,

I saw the news about VyStar Credit Union's recent partnership with NCR Atleos and your expansion plans. It's clear you're on a strong growth trajectory.

At Premier Star Aviation, we specialize in simplifying travel for busy teams. Our private jet brokerage ensures your executives can reach their destinations with ease, offering a straightforward pay-as-you-go service that's available around the clock.

Do you need a quote for any upcoming flights?

Best,

Peter Silks

Founding Partner
"""

example_email_2 = """
Hi Chloe,

Firstly, congratulations on the remarkable visibility Richard Mille has gained with Anant Ambani's choice of timepiece for his pre-wedding festivities.

I’d love to introduce our film studio CFH and see if you have any projects coming up this year that we can help you with?

We specialise in luxury creative content and work with brands such as AMAN, Airelles, and Moët & Chandon.

Would you be interested in seeing some of our work?

Best wishes,

Jessica
"""

example_email_3 = """
Hi James,

Great to see the news about Lego’s decision to find an expanded London HQ near Liverpool Street. The partnership with Fortnite is clearly paying off!

At Viking we’ve worked with multinationals including JP Morgan, Howden and Whole Foods to furnish their new locations with the best office supplies.

We’re just down the street from you, would you be open to a coffee later this week to discuss your upcoming move?

Best wishes,

Madelyn Steele

Head of Commercial
"""

In [7]:
# Pulling example data
dfdict = pd.read_excel("example_data.xlsx", sheet_name=None)
df = dfdict['Sheet1']
example = df.iloc[0]
print(example)

Prospect Name                                                Arnold Palmer
Prospect Role                                                          CEO
Prospect Company Name                                            Anthropic
Prospect Company news    Amazon completes $4B Anthropic investment to a...
Name: 0, dtype: object


In [8]:
# Function to generate personalised email given inputs using OpenAI call
def generate_email(name, role, company_name, company_news):

    email_prompt = f"""
    You are an expert outbound sales agent specialised in email marketing at PerlonAI.
    PerlonAI is a company that uses AI to help businesses send well-researched, hyper-personalized cold emails at scale.
    The following information in triple ***'s is on PerlonAI's website:
    ***PerlonAI: {perlon_description}***
    Write a cold email to {name}, who is the {role} at {company_name}, advertising PerlonAI's offerings.
    The following information in triple ***'s details recent news about {name}'s company, {company_name}. Use this information to personalise the email if applicable.
    ***{company_name} News: {company_news}***
    You must always use a professional but friendly tone.
    You must always sign off as Brent Rohner, Co-Founder of PerlonAI.
    Your email must not exceed 6 sentences. 
    Below are some cold email templates to guide you. 
    Example 1: {example_email_1}
    Example 2: {example_email_2}
    Example 3: {example_email_3}
    """

    chat_completion = client.chat.completions.create(
        model="gpt-4-turbo-2024-04-09",
        messages=[{"role":"system", "content": email_prompt}]
    )
    cold_email = chat_completion.choices[0].message.content

    return cold_email

In [8]:
test = generate_email(example['Prospect Name'], example['Prospect Role'], example['Prospect Company Name'], example['Prospect Company news'])
print(test)

Subject: Enhance Your Outreach with PerlonAI's Hyper-Personalized Cold Emails

Hi Arnold,

Congratulations on Anthropic's recent advancements in generative AI and the unveiling of new challenging models! 

At PerlonAI, we specialise in helping companies like Anthropic send well-researched, hyper-personalized cold emails at scale. Our AI technologies are custom-built to increase email opens by 86% and generate 5X more replies compared to standard templates.

Would you be interested in learning more about how PerlonAI can help Anthropic scale its outreach efforts effectively? 

Best regards,

Brent Rohner
Co-Founder
PerlonAI


In [11]:
# Testing Claude models 
def anthropic_generate_email(name, role, company_name, company_news):

    email_prompt = f"""
    You are an expert outbound sales agent specialised in email marketing at PerlonAI.
    PerlonAI is a company that uses AI to help businesses send well-researched, hyper-personalized cold emails at scale.
    The following information in triple ***'s is on PerlonAI's website:
    ***PerlonAI: {perlon_description}***
    Write a cold email to {name}, who is the {role} at {company_name}, advertising PerlonAI's offerings.
    The following information in triple ***'s details recent news about {name}'s company, {company_name}. Use this information to personalise the email if applicable.
    ***{company_name} News: {company_news}***
    You must always use a professional but friendly tone.
    You must always sign off as Brent Rohner, Co-Founder of PerlonAI.
    Your email must not exceed 6 sentences. 
    Below are some cold email templates to guide you. 
    Example 1: {example_email_1}
    Example 2: {example_email_2}
    Example 3: {example_email_3}
    """

    message = anthropic_client.messages.create(
        model="claude-3-opus-20240229",
        max_tokens=300,
        temperature=0.0,
        system=email_prompt,
        messages=[
            {'role':'user', 'content': 'generate the email!'}
        ]
    )
    
    cold_email = message.content[0].text
    return cold_email

Subject: Hyper-Personalized Cold Emails to Fuel Anthropic's Growth

Hi Arnold,

Congratulations on Anthropic's recent $4B investment from Amazon and the unveiling of your new AI models. As you continue to challenge Big Tech, I believe PerlonAI can help accelerate your growth by revolutionizing your email outreach.

Our AI-powered platform enables you to send hyper-personalized cold emails at scale, leading to a 500% increase in replies and an 86% boost in open rates compared to traditional templates. By automating the research, crafting, and sending of emails directly from your team's mailboxes, PerlonAI saves valuable time and ensures your messages bypass spam filters effectively.

I'd love to schedule a demo to show you how PerlonAI can transform Anthropic's email prospecting efforts and support your ongoing success. Let me know if you have any questions or if there's a convenient time for a quick call.

Best regards,

Brent Rohner
Co-Founder of PerlonAI


In [ ]:
test = anthropic_generate_email(example['Prospect Name'], example['Prospect Role'], example['Prospect Company Name'], example['Prospect Company news'])
print(test)

In [21]:
# Generating all example emails 
chatgpt_emails = []
claude_emails = []

for i in tqdm(range(len(df))):
    example = df.iloc[i]
    gpt_email = generate_email(example['Prospect Name'], example['Prospect Role'], example['Prospect Company Name'], example['Prospect Company news'])
    claude_email = anthropic_generate_email(example['Prospect Name'], example['Prospect Role'], example['Prospect Company Name'], example['Prospect Company news'])
    chatgpt_emails.append(gpt_email)
    claude_emails.append(claude_email)

# Saving generated emails to csv 
results_dict = {"gpt_emails": chatgpt_emails, "claude_emails": claude_emails}
results_df = pd.DataFrame(results_dict)
results_df.to_csv("results.csv")

/var/folders/0c/bxf3tpv51tq6yhtn2r7wsxb00000gn/T/ipykernel_3341/2179649652.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(len(df))):


  0%|          | 0/7 [00:00<?, ?it/s]

### Brief Cost Analysis
- Claude: input=$15/1m tokens, output=$75/1m tokens
- OpenAI: input=$10/1m tokens, output=$30/1m tokens

In [29]:
# Function to create prompt (to count input tokens)
def create_prompt(name, role, company_name, company_news):
    email_prompt = f"""
    You are an expert outbound sales agent specialised in email marketing at PerlonAI.
    PerlonAI is a company that uses AI to help businesses send well-researched, hyper-personalized cold emails at scale.
    The following information in triple ***'s is on PerlonAI's website:
    ***PerlonAI: {perlon_description}***
    Write a cold email to {name}, who is the {role} at {company_name}, advertising PerlonAI's offerings.
    The following information in triple ***'s details recent news about {name}'s company, {company_name}. Use this information to personalise the email if applicable.
    ***{company_name} News: {company_news}***
    You must always use a professional but friendly tone.
    You must always sign off as Brent Rohner, Co-Founder of PerlonAI.
    Your email must not exceed 6 sentences. 
    Below are some cold email templates to guide you. 
    Example 1: {example_email_1}
    Example 2: {example_email_2}
    Example 3: {example_email_3}
    """
    return email_prompt

In [34]:
# Cost analysis 
results_df = pd.read_csv("results.csv")

encoding = tiktoken.get_encoding("cl100k_base")

input_tokens = []
gpt_output_tokens = []
claude_output_tokens = []

# Counting num input tokens for each prompt
for i in range(len(df)):
    example = df.iloc[i]
    p = create_prompt(example['Prospect Name'], example['Prospect Role'], example['Prospect Company Name'], example['Prospect Company news'])
    num_tokens = len(encoding.encode(p))
    input_tokens.append(num_tokens)

# Counting num output tokens for gpt & claude emails
for i in range(len(results_df)):
    email = results_df.iloc[i]
    gpt_tokens = encoding.encode(email['gpt_emails'])
    gpt_output_tokens.append(len(gpt_tokens))
    claude_tokens = encoding.encode(email['claude_emails'])
    claude_output_tokens.append(len(claude_tokens))

print(input_tokens)
print(gpt_output_tokens)
print(claude_output_tokens)

[828, 848, 824, 820, 835, 819, 819]
[130, 152, 139, 138, 151, 130, 141]
[188, 187, 155, 158, 188, 189, 158]


In [40]:
# Calculating cost per email
claude_email_costs = []
gpt_email_costs = []

for i in range(len(input_tokens)):
    claude_cost = input_tokens[i]*15/1000000 + claude_output_tokens[i]*75/1000000
    claude_email_costs.append(claude_cost)
    gpt_cost = input_tokens[i]*10/1000000 + gpt_output_tokens[i]*30/1000000
    gpt_email_costs.append(gpt_cost)

print("The average cost per claude email is : $", round(statistics.mean(claude_email_costs),3))
print("The average cost per gpt email is: $", round(statistics.mean(gpt_email_costs),3))

The average cost per claude email is : $ 0.026
The average cost per gpt email is: $ 0.012


### Refining Prompt Template
- selected gpt for cheap $
- creating email subject
- incorporating linkedin post information
- max 75 words
- subject line, less than 5 words
- added subject line examples, [20 Cold Email Subject Line Examples — That Actually Get Responses](https://www.brafton.co.uk/blog/email-marketing/20-cold-email-subject-lines-that-actually-get-responses/)
- moving recipient information into user content 

In [9]:
# Example subject lines 
example_sl_1 = "Let's talk about email sales"
example_sl_2 = "Have you solved your sales problem yet?"
example_sl_3 = "Essential resources to help you with email sales"

In [10]:
# Function to create refined prompt (to count input tokens)
def create_refined_prompt(name, role, company_name, company_news):
    email_prompt = f"""
    You are an expert outbound sales agent specialised in email marketing at PerlonAI.
    PerlonAI is a company that uses AI to help businesses send well-researched, hyper-personalized cold emails at scale.
    The following information in triple ***'s is on PerlonAI's website:
    ***PerlonAI: {perlon_description}***
    Write a cold email with a subject line to be sent to {name}, who is the {role} at {company_name}, advertising PerlonAI's offerings.
    Your subject line must be less than 5 words.
    Your email must be less than 75 words.
    You must always sign off as Brent Rohner, Co-Founder of PerlonAI.
    The following information in triple ***'s details recent news about {name}'s company, {company_name}. Use this information to personalise the email if applicable.
    ***{company_name} News: {company_news}***
    Here are some example subject lines:
    Example Subject Line 1: {example_sl_1}
    Example Subject Line 2: {example_sl_2}
    Example Subject Line 3: {example_sl_3}
    Here are some example cold emails:
    Example Email 1: {example_email_1}
    Example Email 2: {example_email_2}
    Example Email 3: {example_email_3}
    """
    return email_prompt

In [40]:
# Function to generate personalised email given inputs, refined template
def generate_email_refined(name, role, company_name, company_news):

    email_prompt = create_refined_prompt(name, role, company_name, company_news)

    chat_completion = client.chat.completions.create(
        model="gpt-4-turbo-2024-04-09",
        messages=[{"role":"system", "content": email_prompt}]
    )
    cold_email = chat_completion.choices[0].message.content

    return cold_email

In [42]:
test = generate_email_refined(example['Prospect Name'], example['Prospect Role'], example['Prospect Company Name'], example['Prospect Company news'])
print(test)

Subject Line: Boost Your Email Impact

Hi Arnold,

Congratulations on Anthropic's recent achievements with Amazon and your AI innovations! At PerlonAI, we specialize in using AI to enhance email campaigns, ensuring each message is unique and perfectly tailored.

Given your growth, maximizing your outreach’s efficiency and response could be pivotal. Interested in a quick demo to see how we can help?

Best regards,

Brent Rohner

Co-Founder, PerlonAI


In [11]:
# Function to generate personalised email, moving inputs into user content field
def generate_email_refined_v2(name, role, company_name, company_news):

    email_prompt = f"""
    You are an expert outbound sales agent specialised in email marketing at PerlonAI.
    PerlonAI is a company that uses AI to help businesses send well-researched, hyper-personalized cold emails at scale.
    The following information in triple ***'s is on PerlonAI's website:
    ***PerlonAI: {perlon_description}***
    Write a personalised cold email with a subject line to the specified person, advertising PerlonAi's offerings.
    Your subject line must be less than 5 words.
    Your email must be less than 75 words.
    You must always sign off as Brent Rohner, Co-Founder of PerlonAI.
    Here are some example subject lines:
    Example Subject Line 1: {example_sl_1}
    Example Subject Line 2: {example_sl_2}
    Example Subject Line 3: {example_sl_3}
    Here are some example cold emails:
    Example Email 1: {example_email_1}
    Example Email 2: {example_email_2}
    Example Email 3: {example_email_3}
    """

    user_input = f"""
    Write an email to {name}, who is the {role} at {company_name}. Here is some recent news about {company_name}: {company_news}
    """

    chat_completion = client.chat.completions.create(
        model="gpt-4-turbo-2024-04-09",
        messages=[{"role":"system", "content": email_prompt}, {"role":"user", "content":user_input}]
    )
    cold_email = chat_completion.choices[0].message.content

    return cold_email

In [18]:
test = generate_email_refined_v2(example['Prospect Name'], example['Prospect Role'], example['Prospect Company Name'], example['Prospect Company news'])
print(test)

Subject Line: Elevate Your AI Email Strategy

Hi Arnold,

Congratulations on Anthropic's recent milestones with Amazon and your innovative AI models! I'm Brent Rohner from PerlonAI, where we enhance email outreach through AI-driven personalization.

Our technology could synergistically augment your customer engagement, increasing open rates and replies significantly. Interested in a quick demo to see the potential uplift in action?

Best regards,

Brent Rohner
Co-Founder, PerlonAI


### LLM Evaluation 
- guidelines to evaluate cold email 
- assess subject line, humanness, relevance, persuasiveness

In [12]:
# Function to evaluate a given email based on above metrics
def evaluate_email(email):
    model = ChatOpenAI(model_name="gpt-3.5-turbo")  
    response_schemas = [
        ResponseSchema(name="subject_line", description="Does the subject line sound convincing, out of 5?"),
        ResponseSchema(name="humanness", description="Does the email sound human, out of 5?"),
        ResponseSchema(name="relevance", description="Is the email relevant, out of 5?"),    
        ResponseSchema(name="persuasiveness", description="Is the email persuasive, out of 5?"),        
    ]
    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
    format_instructions = output_parser.get_format_instructions()
    prompt = PromptTemplate(
        template="""You are an expert at writing cold emails that generate leads. 
        Evaluate following cold email and subject line, considering the subject line and the humanness, relevance and persuasiveness of the email.
        If the subject line does not exist or it sounds like it would trigger spam filters, set the subject_line to 0. 
        Format instructions: {format_instructions}
        Cold Email: {cold_email}""",
        input_variables=["cold_email"],
        partial_variables={"format_instructions": format_instructions},
    )
    chain = prompt | model | output_parser
    result = chain.invoke({"cold_email": email})
    return result
    

In [19]:
bad_email = """Subject Line: Great Product
Hi Dave, buy my product please. 
Thanks"""
bad_email_evaluation = evaluate_email(bad_email)
print(bad_email)
print(bad_email_evaluation)

test_evaluation = evaluate_email(test)
print(test)
print(test_evaluation)

Subject Line: Great Product
Hi Dave, buy my product please. 
Thanks
{'subject_line': '0', 'humanness': '2', 'relevance': '1', 'persuasiveness': '2'}
Subject Line: Elevate Your AI Email Strategy

Hi Arnold,

Congratulations on Anthropic's recent milestones with Amazon and your innovative AI models! I'm Brent Rohner from PerlonAI, where we enhance email outreach through AI-driven personalization.

Our technology could synergistically augment your customer engagement, increasing open rates and replies significantly. Interested in a quick demo to see the potential uplift in action?

Best regards,

Brent Rohner
Co-Founder, PerlonAI
{'subject_line': '4', 'humanness': '5', 'relevance': '5', 'persuasiveness': '4'}


### ReAct Framework
- testing ReAct agent framework, [LangChain ReAct](https://python.langchain.com/docs/modules/agents/agent_types/react/), [Arize.com](https://arize.com/blog-course/react-agent-llm/)
- using the evaluator function as a tool

*needs to be reworked, agent/tool setup is unclear/nonsensical*

In [45]:
#instruction prompt for agent 
agent_instructions = f"""
    You are an expert outbound sales agent specialised in email marketing at PerlonAI.
    PerlonAI is a company that uses AI to help businesses send well-researched, hyper-personalized cold emails at scale.
    The following information in triple ***'s is on PerlonAI's website:
    ***PerlonAI: {perlon_description}***
    Write a personalised cold email with a subject line to the specified person, advertising PerlonAi's offerings.
    If your generated email has a score of greater than 15 on the evaluator tool, submit this as your final answer.
    Your subject line must be less than 5 words.
    Your email must be less than 75 words.
    You must always sign off as Brent Rohner, Co-Founder of PerlonAI.
    """
    # Here are some example subject lines:
    # Example Subject Line 1: {example_sl_1}
    # Example Subject Line 2: {example_sl_2}
    # Example Subject Line 3: {example_sl_3}
    # Here are some example cold emails:
    # Example Email 1: {example_email_1}
    # Example Email 2: {example_email_2}
    # Example Email 3: {example_email_3}

# Wrapper function to be passed into agent as a tool
def evaluate(email):
    score = 0
    evaluation = evaluate_email(email)
    score += int(evaluation['subject_line'])
    score += int(evaluation['humanness'])
    score += int(evaluation['relevance'])
    score += int(evaluation['persuasiveness'])
    result = "Email Score: " + str(score) + "/20"
    return result


In [52]:
# Function to call ReAct agent to generate email 
# Uses the evaluator tool to keep checking 
def email_agent(name, role, company_name, company_news):
   
   tools = [
      Tool(
         name="evaluate",
         func=evaluate,
         description="Evaluate your cold email.",
      )
   ]

   input = f"""
      Write an email to {name}, who is the {role} at {company_name}. Here is some recent news about {company_name}: {company_news}. 
      """
   
   prefix = agent_instructions
   prompt = ZeroShotAgent.create_prompt(
      tools, prefix=prefix, input_variables=["input"]
   )
   # print(prompt.template)
   llm_chain = LLMChain(llm=OpenAI(), prompt=prompt)   
   tool_names = [tool.name for tool in tools]
   agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
   agent_executor = AgentExecutor.from_agent_and_tools(
       agent=agent, tools=tools, verbose=True
   )
   result = agent_executor.run(input)
   return result


In [53]:
test = email_agent(example['Prospect Name'], example['Prospect Role'], example['Prospect Company Name'], example['Prospect Company news'])
print(test)


    You are an expert outbound sales agent specialised in email marketing at PerlonAI.
    PerlonAI is a company that uses AI to help businesses send well-researched, hyper-personalized cold emails at scale.
    The following information in triple ***'s is on PerlonAI's website:
    ***PerlonAI: PerlonAI specializes in the creation and distribution of automated, hyper-personalized cold emails designed to significantly improve engagement and response rates over standard email templates. The company employs sophisticated AI models tailored to each customer's unique brand voice and requirements. These AI models automate the entire process, including researching, crafting, and sending emails directly from the user's mailbox.

Key benefits reported by PerlonAI include:
1. **Increased Efficiency**: Template-based campaigns are statistically ineffective; PerlonAI enables personalization at scale without the extensive time investment traditionally required to craft personalized emails, claimi

## Further Improvements

- further investigation of agent methods: better tools? multi-agent?
- improve evaluation mechanism: run generation and evaluation in a loop until threshold t reached
- find better email templates
- define branding / voice better (finetuning? need more data)
- complete ML cycle: deploy & gather more data by testing success rate of generated emails 